In [ ]:
pip install torch torchvision transformers datasets



In [ ]:
# Instalar las dependencias necesarias
!pip install datasets torch torchvision huggingface_hub

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset
from huggingface_hub import login

# Autenticarse en Hugging Face
login(token='<TOKEN HUGGING FACE>')  # Reemplaza 'YOUR_HUGGING_FACE_TOKEN' con tu token

# Cargar los datasets de Hugging Face
cropnet_data = load_dataset('CropNet/CropNet', name='WRF-HRRR Computed Dataset')
usda_data = load_dataset('CropNet/CropNet', name='USDA Crop Dataset')

# Revisar la estructura del dataset
print(cropnet_data)
print(usda_data)

# Preparar los datos (ajusta esto según la estructura específica de tus datos)
def prepare_data(cropnet_dataset, usda_dataset):
    features = []
    labels = []
    for entry, usda_entry in zip(cropnet_dataset['train'], usda_dataset['train']):
        features.append([
            entry['temperature'],
            entry['precipitation'],
            entry['humidity'],
            entry['soil_moisture'],
            usda_entry['crop_type'],
            usda_entry['soil_type']
        ])  # Ejemplo de características climáticas y de suelo
        labels.append(entry['yield'])
    return {'features': features, 'labels': labels}

data = prepare_data(cropnet_data, usda_data)

# Crear un DataLoader
crop_dataset = CropDataset(data['features'], data['labels'])
data_loader = DataLoader(crop_dataset, batch_size=32, shuffle=True)

# Definir el modelo de red neuronal
class CropYieldPredictor(nn.Module):
    def __init__(self):
        super(CropYieldPredictor, self).__init__()
        self.fc1 = nn.Linear(6, 128)  # Ajusta el tamaño de la entrada según tus datos
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = CropYieldPredictor()

# Definir el optimizador y la función de pérdida
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entrenar el modelo
def train_model(dataloader):
    model.train()
    for epoch in range(10):  # Ajusta el número de épocas según sea necesario
        for batch in dataloader:
            features, labels = batch
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Entrenar el modelo con los datos cargados
print("Training on CropNet dataset")
train_model(data_loader)

# Guardar el modelo entrenado
torch.save(model.state_dict(), 'crop_yield_predictor.pth')

print("Model trained and saved successfully!")


In [ ]:
# Instalar las bibliotecas necesarias
!pip install datasets
!pip install mysql-connector-python

import pandas as pd
import mysql.connector
from datasets import load_dataset

# Cargar el dataset de Hugging Face
dataset = load_dataset("cannlytics/cannabis_tests", trust_remote_code=True)

# Inspeccionar las divisiones disponibles
print(dataset)

# Asegúrate de usar la división correcta basada en la inspección
# Aquí suponemos que la división se llama 'details'
df = pd.DataFrame(dataset['details'])

# Guardar el DataFrame en un archivo CSV
csv_file = '/content/cannabis_tests.csv'
df.to_csv(csv_file, index=False)



In [ ]:
# Instalar las bibliotecas necesarias
!pip install datasets
!pip install mysql-connector-python

import pandas as pd
import mysql.connector
from datasets import load_dataset

# Cargar el dataset de Hugging Face
dataset = load_dataset("cannlytics/cannabis_tests", trust_remote_code=True)

# Asegúrate de usar la división correcta basada en la inspección
# Aquí suponemos que la división se llama 'details'
df = pd.DataFrame(dataset['details'])

# Imprimir los nombres de las columnas
print(df.columns)

# Reemplazar NaN por None
df = df.where(pd.notnull(df), None)

# Configurar la conexión a la base de datos MySQL
db_config = {
    'user': 'USER BBDD',
    'password': 'PASSWORD BBDD',
    'host': 'URL BBDD',
    'port': 3336,
    'database': 'NAME BBDD'
}

# Conectar a la base de datos MySQL
cnx = mysql.connector.connect(**db_config)
cursor = cnx.cursor()

# Generar la consulta SQL para crear la tabla
table_name = "cannabis_tests"
columns = df.columns

# Crear la parte de la consulta que define las columnas
create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} (id INT AUTO_INCREMENT PRIMARY KEY, "
for col in columns:
    create_table_query += f"{col} TEXT, "
create_table_query = create_table_query.rstrip(', ') + ");"

# Ejecutar la consulta para crear la tabla
cursor.execute(create_table_query)

# Generar la consulta SQL para insertar datos
insert_query = f"INSERT INTO {table_name} ("
insert_query += ", ".join(columns)
insert_query += ") VALUES ("
insert_query += ", ".join(["%s"] * len(columns))
insert_query += ")"

# Insertar los datos en la tabla
for i, row in df.iterrows():
    values = [None if pd.isna(value) else value for value in row]
    cursor.execute(insert_query, values)

# Confirmar los cambios en la base de datos
cnx.commit()

# Cerrar la conexión
cursor.close()
cnx.close()

print("Datos insertados correctamente en la tabla MySQL.")
